In [7]:
# Installing fuzzywuzzy module
!pip3 install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
professors = pd.read_csv("/content/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

In [9]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [12]:
#get all unique values in the country column
countries = professors['Country'].unique()
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

### 1. The first inconsistency in the data is that we find 'Germany' and 'germany'

In [13]:
# Convert all to lower case
professors['Country'] = professors['Country'].str.lower()

### 2. The second is the unnecessary white spaces

In [14]:
# Remove trailing white space
professors['Country'] = professors['Country'].str.strip()

### 3. It does look like there is another inconsistency: 'southkorea' and 'south korea' should be the same.

In bigger datasets, we can not correct all errors by hand. So, we can use `fuzzywuzzy` package

In [17]:
# Get the top 10 closest matches to 'south korea'
matches = fuzzywuzzy.process.extract("south korea", countries, limit = 10, 
                                     scorer = fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('South Korea', 100),
 ('SouthKorea', 48),
 ('Saudi Arabia', 43),
 ('Norway', 35),
 ('Ireland', 33),
 ('Portugal', 32),
 ('Singapore', 30),
 ('Netherland', 29),
 ('Macau', 25),
 ('USofA', 25)]

''south korea'' and ''southkorea'' to very close. so we can replace all rows in the `Country` column that have a ratio of `> 47` with ''south korea''

In [27]:
# Function to replace rows in the provided column of the provided daatframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
  # get list of unique strings
  strings = df[column].unique()

  # get the top 10 closest matches to out input string
  matches = fuzzywuzzy.process.extract(string_to_match, strings, limit = 10, 
                                       scorer = fuzzywuzzy.fuzz.token_sort_ratio)
  
  # only get matches with a ratio > 90
  close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

  # get the rows of all the close matches in our dataframe
  rows_with_matches = df[column].isin(close_matches)

  # replace all rows with the close matches with the input matches
  df.loc[rows_with_matches, column] = string_to_match

  print("All done")

In [28]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done


In [29]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)